# **TCO Analysis: Electric vs. Diesel Commercial Trucks**

---



**How to Run This Notebook:**

1.  **Upload Data:** Ensure the `TCO_Analysis_Diesel_vs_EV SQL.csv` file is uploaded to your Google Colab session's `/content/` directory. (Click the folder icon on the left sidebar -> upload icon -> select your CSV).
2.  **Run All Cells:** Go to `Runtime` -> `Run all` in the Colab menu.
3.  **Review Output:** Examine the printed dataframes and interactive charts for insights into the TCO analysis.

**Introduction**

- **Project Goal:** To perform a comprehensive 5-year Total Cost of Ownership (TCO) analysis comparing Electric Vehicles (EVs) and Diesel trucks for commercial fleet operations.

- **Data Source:** `TCO_Analysis_Diesel_vs_EV SQL.csv `(exported from SQL Server after initial data structuring).

- **Methodology:** The analysis involves loading and preparing data from the SQL export, engineering key cost features, calculating the 5-year TCO for each vehicle, and visualizing the cost breakdown and key drivers.

- **Outcome:** Provide data-driven insights to support strategic fleet management decisions, identifying which vehicle type offers better long-term financial viability.

---



## **1. Setup and Imports**

This section imports all necessary Python libraries for data manipulation, numerical operations, interactive plotting, and machine learning (for feature importance). Display options for Pandas DataFrames are also configured for better readability.

In [45]:
# 1. Setup and Imports

import warnings
warnings.filterwarnings('ignore') # Suppress minor warnings for cleaner output

import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.ensemble import RandomForestRegressor

# Configure Pandas display options for better readability of DataFrames
pd.set_option('display.max_columns', 200) # Show up to 200 columns
pd.set_option('display.width', 180)     # Adjust display width

## **2. Data Loading and Reshaping from SQL Export**

This code cell defines a function `load_and_reshape_sql_export` that reads the `TCO_Analysis_Diesel_vs_EV SQL.csv` file. This CSV is the result of a `CROSS JOIN` operation in SQL, which means global assumptions are repeated for each vehicle.

The function intelligently reshapes this "long" format into two distinct Python objects:

- `df`: A DataFrame containing unique vehicle specifications.

- `assumptions`: A dictionary containing unique global assumptions (e.g., fuel prices, annual distance).

The column names are explicitly assigned here to ensure correct parsing of the SQL-exported CSV, which typically does not include headers.

In [46]:
# 2. Data Loading and Reshaping (from SQL Export)


def load_and_reshape_sql_export(path):
    """
    This function loads the CSV exported from SQL and reshapes it
    from a long 'cross-join' format into two clean objects: a vehicle DataFrame
    and an assumptions dictionary.
    """
    try:
        # Read without header assumption, assuming first row is data based on typical SQL export
        df_long = pd.read_csv(path, header=None)
    except FileNotFoundError:
        raise FileNotFoundError(f"❌ CRITICAL ERROR: File not found at '{path}'. Please ensure the CSV is uploaded.")

    # Manually assign column names based on the expected data structure from your SQL CROSS JOIN
    # This list must match the order of columns in your exported CSV exactly.
    df_long.columns = [
        'vehicle_id', 'vehicle_name', 'ex_showroom_price', 'charger_cost',
        'road_tax_reg_rate', 'battery_capacity_kwh', 'efficiency',
        'annual_maintenance_rate', 'annual_tyre_cost', 'resale_value_rate',
        'assumption_id', 'assumption_name', 'value'
    ]

    # --- A. Extract and create the clean vehicles DataFrame ---
    # Select only the vehicle-specific columns and drop duplicates to get unique vehicles
    vehicle_cols = ['vehicle_id', 'vehicle_name', 'ex_showroom_price', 'charger_cost',
                    'road_tax_reg_rate', 'battery_capacity_kwh', 'efficiency',
                    'annual_maintenance_rate', 'annual_tyre_cost', 'resale_value_rate']
    df_vehicles = df_long[vehicle_cols].drop_duplicates().reset_index(drop=True)
    df_vehicles['Vehicle_Type'] = np.where(df_vehicles['vehicle_name'].str.contains('Diesel'), 'Diesel', 'EV')


    # --- B. Extract and create the assumptions dictionary ---
    # Select assumption columns and drop duplicates to get unique assumptions
    assumptions_df = df_long[['assumption_name', 'value']].drop_duplicates()
    assumptions = assumptions_df.set_index('assumption_name')['value'].to_dict()

    print("✅ SQL-exported data successfully loaded and reshaped.")
    return df_vehicles, assumptions

# Define the path to the SQL-exported CSV file (ensure it's uploaded to Colab /content/)
DATA_PATH = '/content/TCO_Analysis_Diesel_vs_EV SQL.csv'
df, assumptions = load_and_reshape_sql_export(DATA_PATH)

✅ SQL-exported data successfully loaded and reshaped.


## **3. Feature Engineering: Calculating Cost Components**

In this section, we apply the financial model's logic to create all the necessary cost components for the 5-year TCO calculation. This involves:

- Converting assumption values to numeric types.

- Calculating initial purchase price, fuel costs (differently for diesel and EV), annual maintenance, potential subsidies, and estimated resale value.

In [47]:
# 3. Feature Engineering: Create model-ready columns

print("\n--- Engineering features for TCO model ---")

# Ensure all assumption values are numeric for calculations
for key, value in assumptions.items():
    assumptions[key] = float(value)

# Calculate initial purchase price
df['Purchase_Price'] = df['ex_showroom_price'] + df['charger_cost']

# Calculate fuel cost per kilometer (different logic for Diesel vs. EV)
df['Fuel_Cost_per_km'] = np.where(
    df['Vehicle_Type'] == 'Diesel',
    assumptions['Diesel_Price_Per_Liter'] / df['efficiency'], # Diesel: Price per liter / km/liter
    assumptions['Electricity_Price_Per_kWh'] * df['efficiency'] # EV: Price per kWh * kWh/km
)

# Assign annual distance from assumptions
df['Annual_Km'] = assumptions['Annual_Distance_KM']

# Calculate annual maintenance cost
df['Maintenance_per_year'] = df['ex_showroom_price'] * df['annual_maintenance_rate']

# Calculate potential subsidies for EVs (0 for Diesel)
df['Subsidy'] = np.where(
    df['Vehicle_Type'] == 'EV',
    (df['ex_showroom_price'] * assumptions['FAME_Subsidy_Cap_Rate']) + assumptions['State_Subsidy_Cap'],
    0
)

# Calculate estimated resale value after 5 years
df['Resale_Value_after_5yrs'] = df['Purchase_Price'] * df['resale_value_rate']

# Battery replacement cost is set to 0 as per project scope (not explicitly provided in inputs)
df['Battery_Replacement_Cost'] = 0

print("✅ Feature engineering complete.")
print("\nPreview of key engineered features:")
display(df[['vehicle_name', 'Purchase_Price', 'Fuel_Cost_per_km', 'Maintenance_per_year', 'Subsidy']].head())


--- Engineering features for TCO model ---
✅ Feature engineering complete.

Preview of key engineered features:


,vehicle_name,Purchase_Price,Fuel_Cost_per_km,Maintenance_per_year,Subsidy
0,Diesel Tipper,5500000.0,65.773333,220000.0,0.0
1,Propel 470 MEV,20000000.0,15.300000,285000.0,3850000.0
2,Olectra Meghaetron,16500000.0,13.600000,232500.0,3325000.0
3,Tata Prima E.28k,17000000.0,14.025000,240000.0,3400000.0


## **4. Calculating 5-Year Total Cost of Ownership (TCO)**

This section defines and applies a function to compute the comprehensive 5-year TCO for each vehicle, consolidating all engineered cost components.

In [48]:
# 4. TCO Calculation

def compute_5yr_tco(row):
    """
    Calculates the 5-year Total Cost of Ownership for a given vehicle row.
    """
    total_fuel_cost_5yrs = row['Fuel_Cost_per_km'] * row['Annual_Km'] * 5
    total_maintenance_cost_5yrs = (row['Maintenance_per_year'] + row['annual_tyre_cost']) * 5

    # Gross cost includes purchase, fuel, maintenance, and battery replacement
    gross_cost = row['Purchase_Price'] - row['Subsidy'] + total_fuel_cost_5yrs + total_maintenance_cost_5yrs + row['Battery_Replacement_Cost']

    # Net TCO is gross cost minus the resale value (which acts as a credit)
    net_tco = gross_cost - row['Resale_Value_after_5yrs']
    return net_tco

# Apply the TCO calculation function to each row of the DataFrame
df['TCO_5yr'] = df.apply(compute_5yr_tco, axis=1)

print('\n--- 5-Year TCO computed successfully ---')
print("Vehicles ranked by 5-Year TCO:")
display(df[['vehicle_name', 'Purchase_Price', 'TCO_5yr']].sort_values('TCO_5yr'))


--- 5-Year TCO computed successfully ---
Vehicles ranked by 5-Year TCO:


,vehicle_name,Purchase_Price,TCO_5yr
2,Olectra Meghaetron,16500000.0,14072500.0
3,Tata Prima E.28k,17000000.0,14505625.0
1,Propel 470 MEV,20000000.0,16817500.0
0,Diesel Tipper,5500000.0,21099000.0


## **5. Visualizing TCO Distribution: EV vs Diesel**

This visualization provides a clear comparative summary of the 5-year TCO between Electric and Diesel vehicles using an interactive violin plot. The violin plot shows the distribution, density, and median TCO for each vehicle type.

In [49]:
# 5. Visualizations: TCO Distribution

print("\n--- Comparative Summary: EV vs Diesel ---")
# Aggregate summary statistics for TCO by Vehicle Type
comp_summary = df.groupby('Vehicle_Type').agg(
    Number_of_Vehicles=('TCO_5yr','size'),
    Mean_TCO_5yr=('TCO_5yr','mean'),
    Median_TCO_5yr=('TCO_5yr','median'),
    Mean_Purchase_Price=('Purchase_Price','mean'),
    Mean_Resale_Value=('Resale_Value_after_5yrs','mean')
).reset_index()
display(comp_summary)

# Create an interactive Violin Plot
fig_violin = px.violin(df, x='Vehicle_Type', y='TCO_5yr', color='Vehicle_Type',
                box=True, points='all', hover_data=['vehicle_name', 'Purchase_Price'],
                title='<b>TCO 5-year Distribution: EV vs Diesel (from SQL Data)</b>',
                labels={'TCO_5yr': '5-Year Total Cost of Ownership (INR)', 'Vehicle_Type': ''})
fig_violin.update_layout(showlegend=False) # Hide legend as color is redundant with x-axis
fig_violin.show()


--- Comparative Summary: EV vs Diesel ---


,Vehicle_Type,Number_of_Vehicles,Mean_TCO_5yr,Median_TCO_5yr,Mean_Purchase_Price,Mean_Resale_Value
0,Diesel,1,21099000.0,21099000.0,5.500000e+06,1.100000e+06
1,EV,3,15131875.0,14505625.0,1.783333e+07,4.458333e+06


## **6. Visualizing TCO Cost Component Breakdown**

This stacked bar chart breaks down the total 5-year TCO for each individual vehicle into its contributing components: initial net cost (purchase - subsidy), 5-year fuel, 5-year maintenance/tyre, and resale credit. This helps in understanding where the costs are allocated.

In [50]:
# 6. Visualizations: Cost Component Breakdown

# Calculate 5-year totals for each cost component
df['Initial_Cost_Net'] = df['Purchase_Price'] - df['Subsidy']
df['Fuel_5yr'] = df['Fuel_Cost_per_km'] * df['Annual_Km'] * 5
df['Maint_Tyre_5yr'] = (df['Maintenance_per_year'] + df['annual_tyre_cost']) * 5
df['Resale_Credit'] = -1 * df['Resale_Value_after_5yrs'] # Represent resale as a negative cost (credit)

# Melt the DataFrame to prepare for stacked bar chart (long format needed)
comp_cols = ['vehicle_name', 'Initial_Cost_Net', 'Fuel_5yr', 'Maint_Tyre_5yr', 'Resale_Credit']
comp_df = df[comp_cols].melt(id_vars='vehicle_name', var_name='Cost_Component', value_name='Amount')

# Create an interactive Stacked Bar Chart
fig_bar_components = px.bar(comp_df, x='vehicle_name', y='Amount', color='Cost_Component',
             title='<b>5-Year TCO Cost Component Breakdown (from SQL Data)</b>',
             labels={'Amount': 'Cost/Credit Amount (INR)', 'vehicle_name': 'Vehicle'},
             category_orders={'vehicle_name': df.sort_values('TCO_5yr')['vehicle_name'].tolist()}) # Order by total TCO
fig_bar_components.show()

## **7. Identifying Key Drivers of TCO (Feature Importance)**

A Random Forest Regressor is used here not for prediction, but as an advanced EDA technique to assess the relative importance of different input features in determining the final 5-year TCO. This helps identify the most impactful cost levers.

In [52]:
# 7. Feature Importance for TCO (Random Forest - for insights)

# Define the input features that influence TCO
feature_cols = [
    'ex_showroom_price', 'charger_cost', 'efficiency',
    'annual_maintenance_rate', 'annual_tyre_cost', 'resale_value_rate',
    'Fuel_Cost_per_km' # Our engineered feature, which is a blend of other assumptions
]
# Filter out any columns that might not exist in the DataFrame for robustness
feature_cols = [col for col in feature_cols if col in df.columns]

X = df[feature_cols] # Features (input variables)
y = df['TCO_5yr']     # Target (what we want to explain)

# Ensure there's enough data points for Random Forest (needs at least 2 samples)
if len(X) > 1 and len(y) > 1:
    # Initialize and train a Random Forest Regressor
    rf = RandomForestRegressor(n_estimators=100, random_state=42) # 100 trees, fixed random state for reproducibility
    rf.fit(X, y)

    # Extract feature importances
    imp_df = pd.DataFrame({'feature': X.columns, 'importance': rf.feature_importances_}).sort_values('importance', ascending=False)

    print('\n--- Top Features Influencing 5-Year TCO (from SQL Data) ---')
    display(imp_df)

    # Create an interactive Horizontal Bar Chart for Feature Importance
    fig_importance = px.bar(imp_df, x='importance', y='feature', orientation='h',
                 title='<b>Key Drivers of 5-Year TCO (from SQL Data)</b>',
                 labels={'importance': 'Feature Importance Score', 'feature': 'Input Factor'})
    fig_importance.update_layout(yaxis={'categoryorder':'total ascending'}) # Order bars from lowest to highest importance
    fig_importance.show()
else:
    print("\n⚠️ Not enough data points to compute meaningful Feature Importance (requires at least 2 samples).")


--- Top Features Influencing 5-Year TCO (from SQL Data) ---


,feature,importance
2,efficiency,0.235214
6,Fuel_Cost_per_km,0.220395
0,ex_showroom_price,0.211931
1,charger_cost,0.132526
5,resale_value_rate,0.122872
3,annual_maintenance_rate,0.077062
4,annual_tyre_cost,0.000000


## **8. Key Findings and Insights**

Based on the analysis and visualizations above, here are the critical insights and actionable recommendations for commercial fleet management:

- **Electric Vehicles Offer Lower Long-Term Costs:** The violin plot clearly shows that EVs (e.g., Eicher Pro 2049 EV) have a lower 5-year TCO compared to their diesel counterparts, making them a financially superior choice over the long run.

- **Fuel Costs are Paramount:** The Feature Importance analysis highlights Fuel_Cost_per_km as a dominant factor in TCO. This is visually supported by the large "Fuel_5yr" component in the stacked bar chart, especially for diesel vehicles.

- **Initial Investment vs. Operational Savings:** While EVs have a higher Purchase_Price, their significant savings in Fuel_5yr and Maint_Tyre_5yr (as seen in the stacked bar chart) quickly offset this initial outlay, leading to a lower net TCO.

- **Subsidies Reduce Barriers:** Government subsidies play a crucial role in lowering the "Initial_Cost_Net" for EVs, making them more competitive upfront and accelerating their breakeven point.

- **Resale Value Matters:** The Resale_Credit provides a notable reduction in overall TCO for all vehicles, emphasizing the importance of vehicle longevity and market value.

**Recommendation:** Fleet operators should strategically consider investing in electric trucks. Despite higher upfront costs, the substantial operational savings, driven primarily by lower fuel and maintenance expenses, lead to a more favorable financial outcome over a 5-year period. Leveraging available subsidies is key to mitigating the initial investment barrier.